# KATO Hierarchical Text Generation

**Educational Notebook**: Learn how to use KATO's hierarchical architecture for text generation.

**Prerequisites**: Train patterns first using `training.ipynb`.

## Architecture Overview

This notebook demonstrates **hierarchical text generation** through two complementary processes:

### 1. Bottom-Up Activation (Input → Predictions)

```
User Input: "The cat sat"
     ↓
Tokenize: ["The", " cat", " sat"]
     ↓
node0: Observe tokens → Get predictions (chunk patterns)
     ↓
node1: Observe node0 pattern names → Get predictions (paragraph patterns)
     ↓
node2: Observe node1 pattern names → Get predictions (chapter patterns)
     ↓
node3: Observe node2 pattern names → Get predictions (book patterns)
```

**Result**: All hierarchy levels are activated by the input.

### 2. Top-Down Unraveling (Pattern → Tokens)

```
node3 prediction: PTRN|book_xyz
     ↓
Query MongoDB: Get pattern_data → [node2_pattern_A, node2_pattern_B, ...]
     ↓
For each node2 pattern:
    Query MongoDB → [node1_pattern_X, node1_pattern_Y, ...]
        ↓
    For each node1 pattern:
        Query MongoDB → [node0_pattern_1, node0_pattern_2, ...]
            ↓
        For each node0 pattern:
            Query MongoDB → ["token1", "token2", "token3"]
                ↓
            Decode tokens → "The cat sat on the mat"
```

**Result**: High-level pattern names are recursively unraveled down to actual tokens.

### Cascading Constraint Satisfaction

The combination creates **exponential search space reduction**:

- **node3** says: "We're generating text from 'Moby Dick' context"
- **node2** says: "We're in a chapter about whales"
- **node1** says: "We're in a descriptive paragraph"
- **node0** says: "Next tokens should describe whale behavior"

Each level constrains the levels below, producing coherent, context-aware text.

---

## What You'll Learn

This notebook **exposes all KATO API calls** directly (no library abstractions) so you can see:

1. How to use `observe()`, `observe_sequence()`, `get_predictions()`, `clear_stm()`
2. How to query MongoDB to retrieve pattern structures
3. How to recursively unravel patterns from high-level → tokens
4. How the hierarchy creates coherent, faithful text generation

**Educational Focus**: Understanding KATO's API and hierarchical generation mechanics.

## Setup & Configuration

In [1]:
# # Install required packages
!pip install -q requests datasets transformers # numpy matplotlib tqdm 

In [2]:
# Import required libraries
from tools.kato_client import KATOClient
from transformers import AutoTokenizer
from typing import List, Dict, Any, Tuple
import json

# Note: We're using KATO's get_pattern() API for pattern retrieval
# No direct database access needed - KATO combines ClickHouse + Redis data

print("✓ Imports loaded")

PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


✓ Imports loaded


### Configuration Parameters

**IMPORTANT**: These must match your training configuration!

- `CHUNK_SIZE`: Number of tokens per chunk at node0 (must match training)
- `TOKENIZER_NAME`: HuggingFace tokenizer used during training
- `NODE_IDS`: KATO node identifiers (check your training manifest or MongoDB databases)

In [3]:
# ============================================================================
# CONFIGURATION - MODIFY THESE TO MATCH YOUR TRAINING
# ============================================================================

# Hierarchical configuration (MUST match training!)
# chunk_sizes: How many tokens/patterns to process at each level
# max_predictions: How many predictions in each ensemble sent to next level
CHUNK_SIZES = [8, 8, 8, 8]      # [node0, node1, node2, node3]
MAX_PREDICTIONS = [10, 10, 10, 10]  # [node0, node1, node2, node3]

# Tokenizer (must match training)
TOKENIZER_NAME = "gpt2"  # Options: "gpt2", "bert-base-uncased", "roberta-base", etc.

# Recall threshold (pattern matching strictness)
# Range: 0.0-1.0
#   0.1 = permissive (many matches, lower quality)
#   0.9 = strict (few matches, higher quality)
# Default: 0.6 (balanced)
RECALL_THRESHOLD_DEFAULT = 0.6

# KATO server URL
BASE_URL = "http://kato:8000"

# Node identifiers (MUST match your training configuration)
# Check your ClickHouse/Redis databases to find the correct node_ids
# Format: {node_id}_kato databases in ClickHouse/Redis
NODE_IDS = [
    "node0",  # node0: chunk-level patterns (15 tokens)
    "node1",  # node1: paragraph-level patterns (~225 tokens)
    "node2",  # node2: chapter-level patterns (~3,375 tokens)
    "node3"   # node3: book-level patterns (~50,625 tokens)
]

# Prediction fallback chain (try node3 first, fallback to node2, then node1, then node0)
FALLBACK_LEVELS = [3, 2, 1, 0]

# Validate configuration
if len(CHUNK_SIZES) != len(NODE_IDS):
    raise ValueError(f"CHUNK_SIZES length ({len(CHUNK_SIZES)}) must match NODE_IDS length ({len(NODE_IDS)})")
if len(MAX_PREDICTIONS) != len(NODE_IDS):
    raise ValueError(f"MAX_PREDICTIONS length ({len(MAX_PREDICTIONS)}) must match NODE_IDS length ({len(NODE_IDS)})")

print("✓ Configuration loaded")
print(f"  Chunk sizes: {CHUNK_SIZES}")
print(f"  Max predictions: {MAX_PREDICTIONS}")
print(f"  Tokenizer: {TOKENIZER_NAME}")
print(f"  Node IDs: {NODE_IDS}")

✓ Configuration loaded
  Chunk sizes: [8, 8, 8, 8]
  Max predictions: [10, 10, 10, 10]
  Tokenizer: gpt2
  Node IDs: ['node0', 'node1', 'node2', 'node3']


## Helper Class: TextTokenizer

Simple wrapper around HuggingFace tokenizer for:
1. Tokenizing input text
2. Chunking tokens into fixed-length sequences
3. Decoding tokens back to text

In [4]:
class TextTokenizer:
    """
    Tokenizer wrapper for KATO hierarchical generation.
    
    Uses HuggingFace AutoTokenizer for tokenization and decoding.
    """
    
    def __init__(self, tokenizer_name: str):
        """Initialize with HuggingFace tokenizer."""
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.tokenizer_name = tokenizer_name
        
        # Required for some tokenizers (GPT-2, etc.)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
    
    def tokenize(self, text: str) -> List[str]:
        """
        Tokenize text into list of token strings.
        
        Args:
            text: Input text to tokenize
            
        Returns:
            List of token strings (e.g., ["The", " cat", " sat"])
        """
        # Tokenize using HuggingFace tokenizer
        tokens = self.tokenizer.tokenize(text)
        return tokens
    
    def chunk_tokens(self, tokens: List[str], chunk_size: int) -> List[List[str]]:
        """
        Split tokens into fixed-length chunks.
        
        Args:
            tokens: List of token strings
            chunk_size: Number of tokens per chunk
            
        Returns:
            List of token chunks (last chunk may be shorter)
            
        Example:
            >>> tokens = ["The", " cat", " sat", " on", " the", " mat"]
            >>> chunk_tokens(tokens, chunk_size=3)
            [["The", " cat", " sat"], [" on", " the", " mat"]]
        """
        chunks = []
        for i in range(0, len(tokens), chunk_size):
            chunk = tokens[i:i + chunk_size]
            chunks.append(chunk)
        return chunks
    
    def decode_tokens(self, tokens: List[str]) -> str:
        """
        Decode token strings back to text.
        
        Args:
            tokens: List of token strings
            
        Returns:
            Decoded text string
        """
        # Convert token strings to IDs, then decode
        # This handles subword tokenization correctly
        text = self.tokenizer.convert_tokens_to_string(tokens)
        return text


# Initialize tokenizer
tokenizer = TextTokenizer(TOKENIZER_NAME)
print(f"✓ Tokenizer initialized: {TOKENIZER_NAME}")

# Test tokenization
test_text = "The cat sat on the mat."
test_tokens = tokenizer.tokenize(test_text)
print(f"\nTest tokenization:")
print(f"  Input: {test_text}")
print(f"  Tokens: {test_tokens}")
print(f"  Decoded: {tokenizer.decode_tokens(test_tokens)}")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

✓ Tokenizer initialized: gpt2

Test tokenization:
  Input: The cat sat on the mat.
  Tokens: ['The', 'Ġcat', 'Ġsat', 'Ġon', 'Ġthe', 'Ġmat', '.']
  Decoded: The cat sat on the mat.


## KATO Client Session Cleanup

Why This Is Needed:

* Prevents multiple sessions for the same node
* Clears stale state from previous runs
* Ensures fresh initialization

In [5]:
# ============================================================================
# SESSION CLEANUP - Ensure Clean State
# ============================================================================
# Close any existing KATO sessions to prevent stale state issues.
# This is especially important when re-running cells after errors.

print("Cleaning up old sessions...")

try:
    # Try to close existing node clients
    for node in [node0, node1, node2, node3]:
        try:
            node.close()
            print(f"  ✓ Closed {node.node_id} session")
        except Exception as e:
            # Ignore errors - node may not exist or session already closed
            pass
except NameError:
    # Nodes don't exist yet (first run)
    print("  ℹ No existing nodes to clean up (first run)")

print("✓ Session cleanup complete\n")

Cleaning up old sessions...
  ℹ No existing nodes to clean up (first run)
✓ Session cleanup complete



## KATO Client Initialization

**Educational**: Create one `KATOClient` instance per hierarchical level.

Each client:
- Connects to KATO server
- Has its own isolated session
- Uses `max_pattern_length=0` (prediction mode, not learning)
- Maintains its own Short-Term Memory (STM)

**Note**: We're using KATO for **generation**, not training, so we won't call `learn()`.

In [7]:
# ============================================================================
# KATO CLIENT INITIALIZATION - ONE PER NODE
# ============================================================================

# Create separate KATO clients for each hierarchical level
# max_pattern_length=0: Prediction mode (no auto-learning)
# recall_threshold: Pattern matching strictness
# max_predictions: Number of predictions per ensemble (KATO config)
# process_predictions=True: MUST enable predictions (may be disabled from training)

print("Initializing KATO clients...")

# recall_threshold controls pattern matching strictness:
#   High (0.7-0.9): Strict matching, fewer but higher-quality predictions
#   Medium (0.4-0.6): Balanced (default: 0.6)
#   Low (0.1-0.3): Permissive matching, more predictions (useful for novel inputs)
#
# max_predictions controls ensemble size:
#   - KATO returns top N predictions per call
#   - Entire ensemble sent as ONE event to next level
#   - KATO's pattern matching handles missing/extra symbols gracefully
#   - Higher values: more context but slower, potentially noisy
#   - Lower values: faster but may miss important patterns
#
# process_predictions=True:
#   - CRITICAL: Must be True for generation
#   - During training, this is often set to False to save computation
#   - Explicitly setting True here ensures predictions work regardless of training config

node0 = KATOClient(
    base_url=BASE_URL,
    node_id=NODE_IDS[0],
    max_pattern_length=0,
    recall_threshold=RECALL_THRESHOLD_DEFAULT,
    max_predictions=MAX_PREDICTIONS[0],
    process_predictions=False,  # Enable prediction processing
    timeout=1200
)

node0.update_session_config({
      'use_token_matching': True,  # False = Character-level mode, True = Token-level mode
      'filter_pipeline': ['jaccard'],
      'jaccard_threshold': 0.3,      # Token set overlap
      'jaccard_min_overlap': 2,      # At least 2 shared tokens
      # 'minhash_threshold': 0.1,      # Lower = more recall (try 0.4-0.6)
      'recall_threshold': 0.3,       # Sequence similarity
      'max_predictions': 10,
      'max_candidates_per_stage': 1000,
})

config = node0.get_session_config()
print(f"  ✓ node0: recall={config['recall_threshold']}, max_pred={config['max_predictions']}")

node1 = KATOClient(
    base_url=BASE_URL,
    node_id=NODE_IDS[1],
    max_pattern_length=0,
    recall_threshold=0.1,
    max_predictions=MAX_PREDICTIONS[1],
    process_predictions=False,  # Enable prediction processing
    timeout=1200
)
config = node1.get_session_config()
print(f"  ✓ node1: recall={config['recall_threshold']}, max_pred={config['max_predictions']}")


node2 = KATOClient(
    base_url=BASE_URL,
    node_id=NODE_IDS[2],
    max_pattern_length=0,
    recall_threshold=0.1,
    max_predictions=MAX_PREDICTIONS[2],
    process_predictions=False,  # Enable prediction processing
    timeout=1200
)
config = node2.get_session_config()
print(f"  ✓ node2: recall={config['recall_threshold']}, max_pred={config['max_predictions']}")


node3 = KATOClient(
    base_url=BASE_URL,
    node_id=NODE_IDS[3],
    max_pattern_length=0,
    recall_threshold=0.1,
    max_predictions=MAX_PREDICTIONS[3],
    process_predictions=False,  # Enable prediction processing
    timeout=1200
)
config = node3.get_session_config()
print(f"  ✓ node3: recall={config['recall_threshold']}, max_pred={config['max_predictions']}")


# Store in list for easy iteration
nodes = [node0, node1, node2, node3]

print("\n✓ All KATO clients ready for generation")

Initializing KATO clients...
  ✓ node0: recall=0.3, max_pred=10
  ✓ node1: recall=0.1, max_pred=10
  ✓ node2: recall=0.1, max_pred=10
  ✓ node3: recall=0.1, max_pred=10

✓ All KATO clients ready for generation


In [ ]:
# # Configure nodes for better generation based on validated settings
# node0 uses jaccard filter for token matching
# # Higher nodes use empty filter_pipeline for default pattern matching
# result = node0.update_session_config({      
#     'filter_pipeline': ['jaccard'],      
#     'jaccard_threshold': 0.3,      
#     # Token set overlap      
#     'jaccard_min_overlap': 2,      
#     # At least 2 shared tokens      
#     'recall_threshold': 0.3,       
#     # Sequence similarity (lower for more matches)      
#     'max_predictions': 10,      
#     'use_token_matching': True     
#     # Enable token-level matching for node0
# })

# print("node0:", result)
# # Verify token matching is enabled
# config = node0.get_session_config()
# print(f"  ✓ use_token_matching: {config.get('use_token_matching')}")
# # For node1, node2, node3: Use empty filter_pipeline if there's not a lot of data
# # Jaccard at higher levels can return nothing if patterns are sparse
# result = node1.update_session_config({      'filter_pipeline': [],  
#                                       # Empty = use default pattern matcher      
#                                       'recall_threshold': 0.1,       
#                                       # Very permissive for higher levels      
#                                       'max_predictions': 10,})
# print("node1:", result)result = node2.update_session_config({      
#     'filter_pipeline': [],  # Empty = use default pattern matcher      
#     'recall_threshold': 0.1,       # Very permissive for higher levels      
#     'max_predictions': 10,})print("node2:", result)
# result = node3.update_session_config({      
#     'filter_pipeline': [],  # Empty = use default pattern matcher      
#     'recall_threshold': 0.1,       
#     # Very permissive for higher levels      
#     'max_predictions': 10,})
# print("node3:", result)

## Pattern Retrieval via KATO API

**Educational**: KATO provides a `get_pattern()` API that retrieves learned patterns from its hybrid storage architecture.

### How KATO Stores Patterns

KATO internally uses two databases:
- **ClickHouse**: Stores pattern structure (`pattern_data`, `length`, `tokens`, etc.)
- **Redis**: Stores metadata (`frequency`, `emotives`, additional context)

### Using KATO's API

Instead of directly accessing databases, we use KATO's API:

```python
# Get a pattern via KATO API
result = node.get_pattern(pattern_name)

# Response structure:
{
    "pattern": {
        "status": "okay",
        "pattern": {
            "name": "abc123...",
            "pattern_data": [["child1"], ["child2"], ...],
            "length": 8,
            "frequency": 42,
            "metadata": {...},
            // Combined ClickHouse + Redis data
        }
    }
}
```

### Pattern Data Structure

For **node0**, `pattern_data` contains actual tokens: `[["The"], [" cat"], [" sat"]]`

For **higher levels**, `pattern_data` contains child pattern names from the level below.

**Benefits of using KATO's API**:
- Combines data from both storage systems
- Handles errors and missing patterns gracefully
- Maintains abstraction - storage can change without breaking code
- Provides consistent interface across all levels

In [8]:
# ============================================================================
# HELPER FUNCTION - Pattern Name Prefix Handling
# ============================================================================

def strip_pattern_prefix(pattern_name: str) -> str:
    """
    Strip 'PTRN|' prefix from pattern name if present.
    
    MongoDB stores pattern names WITHOUT the 'PTRN|' prefix in the 'name' field,
    but pattern references in pattern_data and KATO predictions may include it.
    This function ensures consistent lookups.
    
    Args:
        pattern_name: Pattern name, possibly with 'PTRN|' prefix
        
    Returns:
        Pattern name without 'PTRN|' prefix
        
    Examples:
        >>> strip_pattern_prefix('PTRN|abc123')
        'abc123'
        >>> strip_pattern_prefix('abc123')
        'abc123'
    """
    if pattern_name.startswith('PTRN|'):
        return pattern_name[5:]  # Remove 'PTRN|' (5 characters)
    return pattern_name

print("✓ Helper function defined")

✓ Helper function defined


In [9]:
# ============================================================================
# HELPER FUNCTION - STM Inspection for Debugging
# ============================================================================

def inspect_stm(node, node_name: str, verbose: bool = True):
    """
    Get and display current STM (Short-Term Memory) state.
    
    This helper is useful for debugging cascade issues - shows how many
    events are in STM and previews the content.
    
    Args:
        node: KATOClient instance
        node_name: Display name (e.g., "node0", "node1")
        verbose: Whether to print details
        
    Returns:
        List of STM events
        
    Example:
        >>> inspect_stm(node1, "node1")
        node1 STM: 3 events
          STM: ['PTRN|abc', 'PTRN|def', 'PTRN|xyz']
    """
    stm_data = node.get_stm()
    stm = stm_data.get('stm', [])
    
    if verbose:
        print(f"  {node_name} STM: {len(stm)} events")
        if len(stm) > 0:
            print(f"     STM: {stm}")    
    return stm

print("✓ STM inspection helper defined")

✓ STM inspection helper defined


In [10]:
def activate_hierarchy(
    input_text: str,
    verbose: bool = True,
    recall_threshold_overrides: Dict[str, Dict[str, Any]] = None,
    chunk_sizes: List[int] = None,
    max_predictions: List[int] = None
) -> Dict[int, Dict]:
    """
    Activate hierarchical KATO system with input text (bottom-up).
    
    CORRECTED FLOW: For each chunk, cascade predictions through ALL levels.
    
    For each chunk:
        1. Clear node0 STM
        2. Observe chunk at node0 → get predictions
        3. If predictions exist → observe at node1 → get predictions
        4. If predictions exist → observe at node2 → get predictions
        5. If predictions exist → observe at node3 → get predictions
    
    Higher levels (node1, node2, node3) accumulate events across chunks.
    
    CRITICAL: Pattern names must include "PTRN|" prefix when sent to higher levels
    to match training format.
    
    Args:
        input_text: User input text
        verbose: Print intermediate results (including STM inspection)
        recall_threshold_overrides: Per-node recall thresholds
        chunk_sizes: Override CHUNK_SIZES for testing
        max_predictions: Override MAX_PREDICTIONS for testing
    
    Returns:
        Dict mapping level → predictions (from last chunk's cascade)
    """
    # Use global configs if not overridden
    if chunk_sizes is None:
        chunk_sizes = CHUNK_SIZES
    if max_predictions is None:
        max_predictions = MAX_PREDICTIONS
    
    # Validate
    if len(chunk_sizes) != len(nodes):
        raise ValueError(f"chunk_sizes must have {len(nodes)} elements, got {len(chunk_sizes)}")
    if len(max_predictions) != len(nodes):
        raise ValueError(f"max_predictions must have {len(nodes)} elements, got {len(max_predictions)}")
    
    if verbose:
        print(f"\n{'='*80}")
        print("BOTTOM-UP ACTIVATION (Chunk-by-Chunk Cascading)")
        print(f"{'='*80}")
        print(f"Input: {input_text}")
        print(f"Config: chunk_sizes={chunk_sizes}, max_pred={max_predictions}")
    
    # Apply recall_threshold overrides if specified
    if recall_threshold_overrides:
        if verbose:
            print(f"\n--- APPLYING RECALL_THRESHOLD OVERRIDES ---")
        
        for node_name, genes in recall_threshold_overrides.items():
            node_idx = int(node_name.replace('node', ''))
            nodes[node_idx].update_genes(genes)
            
            if verbose and 'recall_threshold' in genes:
                print(f"✓ Updated {node_name} recall_threshold: {genes['recall_threshold']}")
        
        if verbose:
            print()
    
    # Step 1: Tokenize input text
    tokens = tokenizer.tokenize(input_text)
    if verbose:
        print(f"\nTokens ({len(tokens)}): {tokens}")
    
    # Step 2: Chunk tokens using chunk_sizes[0] (node0's chunk size)
    chunks = tokenizer.chunk_tokens(tokens, chunk_sizes[0])
    if verbose:
        print(f"\nChunks ({len(chunks)}) with chunk_size={chunk_sizes[0]}:")
        for idx, chunk in enumerate(chunks):
            print(f"  Chunk {idx}: {chunk}")
    
    # Clear all nodes' STM before starting
    if verbose:
        print(f"\n--- INITIALIZING: Clearing all nodes' STM ---")
    for i, node in enumerate(nodes):
        node.clear_stm()
        if verbose:
            print(f"✓ Cleared node{i} STM")
    
    # Track predictions from last cascade (for return value)
    predictions_0 = {'predictions': []}
    predictions_1 = {'predictions': []}
    predictions_2 = {'predictions': []}
    predictions_3 = {'predictions': []}
    
    # ========================================================================
    # PROCESS EACH CHUNK: CASCADE THROUGH ALL LEVELS
    # ========================================================================
    
    for chunk_idx, chunk in enumerate(chunks):
        if verbose:
            print(f"\n{'='*80}")
            print(f"CHUNK {chunk_idx + 1}/{len(chunks)}: {chunk}")
            print(f"{'='*80}")
        
        # Clear node0 STM for new chunk
        node0.clear_stm()
        if verbose:
            print(f"✓ Cleared node0 STM for new chunk")
        
        # ====================================================================
        # NODE0: Observe chunk → Get predictions
        # ====================================================================
        
        if verbose:
            print(f"\n--- NODE0 ---")
        
        # Build observations: ONE EVENT PER TOKEN
        observations = [{'strings': [token]} for token in chunk]
        
        # Send chunk to node0
        node0.observe_sequence(observations=observations, learn_at_end=False)
        if verbose:
            print(f"✓ Observed {len(chunk)} tokens")
            inspect_stm(node0, "node0")
        
        # Get predictions from node0
        predictions_0 = node0.get_predictions()
        num_preds_0 = len(predictions_0.get('predictions', []))
        
        if verbose:
            print(f"✓ Got {num_preds_0} predictions")
            if num_preds_0 > 0:
                print(f"  Sample predictions:")
                for i, pred in enumerate(predictions_0['predictions'][:3]):
                    print(f"    {i+1}. {pred['name'][:50]}... (conf: {pred.get('confidence', 0):.3f})")
            else:
                print(f"  ⚠ No predictions from node0")
                print(f"     Possible reasons:")
                print(f"       - No patterns in KB match this token sequence")
                print(f"       - recall_threshold too high (current: {RECALL_THRESHOLD_DEFAULT})")
        
        if num_preds_0 == 0:
            if verbose:
                print(f"⚠ No predictions from node0 - stopping cascade for this chunk")
            continue  # No predictions, can't cascade further
        
        # ====================================================================
        # NODE1: Observe node0 predictions → Get predictions
        # ====================================================================
        
        if verbose:
            print(f"\n--- NODE1 ---")
        
        # Send node0 predictions as ONE event to node1
        # CRITICAL: Prepend "PTRN|" to match training format!
        pattern_names_0 = [f"PTRN|{pred['name']}" for pred in predictions_0['predictions']]
        
        if verbose:
            print(f"Sending {len(pattern_names_0)} pattern names as 1 event:")
            for i, name in enumerate(pattern_names_0[:3]):
                print(f"    {i+1}. {name[:50]}...")
        
        node1.observe(strings=pattern_names_0)
        
        if verbose:
            print(f"✓ Observed (1 event)")
            inspect_stm(node1, "node1")
        
        # Get predictions from node1
        predictions_1 = node1.get_predictions()
        num_preds_1 = len(predictions_1.get('predictions', []))
        
        if verbose:
            print(f"✓ Got {num_preds_1} predictions")
            if num_preds_1 > 0:
                print(f"  Sample predictions:")
                for i, pred in enumerate(predictions_1['predictions'][:3]):
                    print(f"    {i+1}. {pred['name'][:50]}... (conf: {pred.get('confidence', 0):.3f})")
            else:
                stm = node1.get_stm().get('stm', [])
                print(f"  ⚠ No predictions from node1")
                print(f"     STM has {len(stm)} event(s)")
                if len(stm) > 0:
                    print(f"     First event: {stm[0][:3]}... ({len(stm[0])} symbols)")
                print(f"     Possible reasons:")
                print(f"       - No node1 patterns in KB match this sequence of node0 patterns")
                print(f"       - recall_threshold too high (current: {RECALL_THRESHOLD_DEFAULT})")
                print(f"       - Pattern names don't match KB format")
        
        if num_preds_1 == 0:
            if verbose:
                print(f"⚠ No predictions from node1 - stopping cascade for this chunk")
            continue  # No predictions, can't cascade further
        
        # ====================================================================
        # NODE2: Observe node1 predictions → Get predictions
        # ====================================================================
        
        if verbose:
            print(f"\n--- NODE2 ---")
        
        # Send node1 predictions as ONE event to node2
        # CRITICAL: Prepend "PTRN|" to match training format!
        pattern_names_1 = [f"PTRN|{pred['name']}" for pred in predictions_1['predictions']]
        
        if verbose:
            print(f"Sending {len(pattern_names_1)} pattern names as 1 event:")
            for i, name in enumerate(pattern_names_1[:3]):
                print(f"    {i+1}. {name[:50]}...")
        
        node2.observe(strings=pattern_names_1)
        
        if verbose:
            print(f"✓ Observed (1 event)")
            inspect_stm(node2, "node2")
        
        # Get predictions from node2
        predictions_2 = node2.get_predictions()
        num_preds_2 = len(predictions_2.get('predictions', []))
        
        if verbose:
            print(f"✓ Got {num_preds_2} predictions")
            if num_preds_2 > 0:
                print(f"  Sample predictions:")
                for i, pred in enumerate(predictions_2['predictions'][:3]):
                    print(f"    {i+1}. {pred['name'][:50]}... (conf: {pred.get('confidence', 0):.3f})")
            else:
                stm = node2.get_stm().get('stm', [])
                print(f"  ⚠ No predictions from node2")
                print(f"     STM has {len(stm)} event(s)")
                if len(stm) > 0:
                    print(f"     First event: {stm[0][:3]}... ({len(stm[0])} symbols)")
                print(f"     Possible reasons:")
                print(f"       - No node2 patterns in KB match this sequence")
                print(f"       - recall_threshold too high")
        
        if num_preds_2 == 0:
            if verbose:
                print(f"⚠ No predictions from node2 - stopping cascade for this chunk")
            continue  # No predictions, can't cascade further
        
        # ====================================================================
        # NODE3: Observe node2 predictions → Get predictions
        # ====================================================================
        
        if verbose:
            print(f"\n--- NODE3 ---")
        
        # Send node2 predictions as ONE event to node3
        # CRITICAL: Prepend "PTRN|" to match training format!
        pattern_names_2 = [f"PTRN|{pred['name']}" for pred in predictions_2['predictions']]
        
        if verbose:
            print(f"Sending {len(pattern_names_2)} pattern names as 1 event:")
            for i, name in enumerate(pattern_names_2[:3]):
                print(f"    {i+1}. {name[:50]}...")
        
        node3.observe(strings=pattern_names_2)
        
        if verbose:
            print(f"✓ Observed (1 event)")
            inspect_stm(node3, "node3")
        
        # Get predictions from node3
        predictions_3 = node3.get_predictions()
        num_preds_3 = len(predictions_3.get('predictions', []))
        
        if verbose:
            print(f"✓ Got {num_preds_3} predictions")
            if num_preds_3 > 0:
                print(f"  Sample predictions:")
                for i, pred in enumerate(predictions_3['predictions'][:3]):
                    print(f"    {i+1}. {pred['name'][:50]}... (conf: {pred.get('confidence', 0):.3f})")
            else:
                stm = node3.get_stm().get('stm', [])
                print(f"  ⚠ No predictions from node3")
                print(f"     STM has {len(stm)} event(s)")
                if len(stm) > 0:
                    print(f"     First event: {stm[0][:3]}... ({len(stm[0])} symbols)")
    
    # Return predictions from last chunk's cascade
    if verbose:
        print(f"\n{'='*80}")
        print("ACTIVATION COMPLETE")
        print(f"{'='*80}")
        print(f"\nFinal prediction counts (from last chunk cascade):")
        print(f"  node0: {len(predictions_0.get('predictions', []))} predictions")
        print(f"  node1: {len(predictions_1.get('predictions', []))} predictions")
        print(f"  node2: {len(predictions_2.get('predictions', []))} predictions")
        print(f"  node3: {len(predictions_3.get('predictions', []))} predictions")
    
    return {
        0: predictions_0,
        1: predictions_1,
        2: predictions_2,
        3: predictions_3
    }

print("✓ activate_hierarchy() function defined")

✓ activate_hierarchy() function defined


### Understanding Hierarchical Activation Flow (CORRECTED)

#### Key Principle: Per-Chunk Cascading

**CRITICAL**: For EACH chunk, predictions cascade through ALL hierarchical levels immediately.

```
Chunk 1 Processing:
  Tokens → node0 → predictions → node1 → predictions → node2 → predictions → node3

Chunk 2 Processing:
  Tokens → node0 → predictions → node1 → predictions → node2 → predictions → node3

Chunk 3 Processing:
  Tokens → node0 → predictions → node1 → predictions → node2 → predictions → node3
```

**NOT** (this would be wrong):
```
❌ All chunks → node0 → THEN all to node1 → THEN all to node2 → THEN all to node3
```

---

#### Detailed Flow for Each Chunk

```python
# For each chunk in input:

1. Clear node0 STM (fresh processing for new chunk)
2. node0.observe(chunk_tokens) → get_predictions()
   └─ If predictions exist:
3.     node1.observe(node0_pattern_names) → get_predictions()
       └─ If predictions exist:
4.         node2.observe(node1_pattern_names) → get_predictions()
           └─ If predictions exist:
5.             node3.observe(node2_pattern_names) → get_predictions()

# Higher levels (node1, node2, node3) accumulate events across chunks
# node0 is cleared between chunks to match training
```

---

#### Why This Matters

**During Training:**
```
Chunk 1: Tokens → node0 learns → Sends 1 pattern name to node1
Chunk 2: Tokens → node0 learns → Sends 1 pattern name to node1
Chunk 3: Tokens → node0 learns → Sends 1 pattern name to node1
...
node1 sees sequence: [name1, name2, name3, ...] → learns patterns
```

**During Generation (must match):**
```
Chunk 1: Tokens → node0 predicts → Sends N pattern names to node1 (ensemble)
Chunk 2: Tokens → node0 predicts → Sends N pattern names to node1 (ensemble)
Chunk 3: Tokens → node0 predicts → Sends N pattern names to node1 (ensemble)
...
node1 sees sequence: [ensemble1, ensemble2, ensemble3, ...] → predicts
```

**Key Insight**: 
- Each chunk cascades immediately through all levels
- Higher levels accumulate events from multiple chunks
- node0 is cleared per-chunk (matches training)
- node1/node2/node3 maintain their STM across chunks (accumulate context)

---

#### Example: 3 Chunks

```
Input: "The cat sat on the mat" 
Chunks: ["The cat sat", "on the mat"]

Chunk 1: ["The", "cat", "sat"]
  node0: observe → predict [A, B, C]
  node1: observe [A, B, C] → predict [X, Y]
  node2: observe [X, Y] → predict [P]
  node3: observe [P] → predict [M]
  
Chunk 2: ["on", "the", "mat"]
  node0: clear, observe → predict [D, E, F]
  node1: observe [D, E, F] → predict [W, Z]  # STM now has 2 events
  node2: observe [W, Z] → predict [Q]        # STM now has 2 events
  node3: observe [Q] → predict [N]            # STM now has 2 events

Final predictions returned: node0=[D,E,F], node1=[W,Z], node2=[Q], node3=[N]
```

**Why clear node0 but not others?**
- node0 processes fixed-length chunks (must match training chunk size)
- node1+ accumulate sequences over time (match training's paragraph/chapter/book accumulation)

---

#### Stopping Cascade Early

If any level returns 0 predictions, the cascade stops for that chunk:

```
Chunk 1:
  node0 → 0 predictions ⚠ STOP (no cascade to higher levels)

Chunk 2:
  node0 → [A, B]
  node1 → 0 predictions ⚠ STOP (no cascade to node2/node3)
```

This prevents propagating "no match" signals up the hierarchy.

## Function 2: Prediction Ensemble with Fallback

**Purpose**: Select predictions from the highest level that has results **with usable 'future' data**.

**Fallback Logic**:
1. Try node3 first (book-level context)
2. If empty or no 'future' data, try node2 (chapter-level context)
3. If empty or no 'future' data, try node1 (paragraph-level context)
4. If empty or no 'future' data, try node0 (chunk-level context)

**IMPORTANT**: Only predictions with **non-empty 'future' field** can be used for generation!

**Why 'future' validation?**: 

KATO predictions contain:
- `'name'`: The matched pattern (what was recognized)
- `'future'`: Predicted next symbols/tokens (what comes next)

For text generation, we need the `'future'` field. Without it, we can't generate predictions - we'd just be unraveling the matched pattern (regenerating training data).

**Example**:
```python
# Prediction WITH 'future' (usable ✓)
{
    'name': 'PTRN|abc123',
    'future': [['PTRN|def456'], ['PTRN|ghi789']],  # ✓ Can generate from this
    'confidence': 0.85
}

# Prediction WITHOUT 'future' (unusable ❌)
{
    'name': 'PTRN|abc123',
    'future': [],  # ❌ Empty - nothing to generate
    'confidence': 0.85
}
```

**Why fallback?**: Novel input may not activate high-level patterns, or high-level patterns may not have 'future' data. Fallback ensures we find predictions with usable 'future' field at some level.

In [11]:
def get_prediction_ensemble(
    all_predictions: Dict[int, Dict],
    fallback_levels: List[int] = [3, 2, 1, 0],
    verbose: bool = True
) -> Tuple[List[Dict], int]:
    """
    Get prediction ensemble from highest available level with usable future data.
    
    Tries each level in fallback_levels order, returning predictions
    from the first level that has non-empty 'future' fields.
    
    IMPORTANT: Only predictions with non-empty 'future' field can be used
    for text generation. The 'future' field contains the predicted next
    pattern names or tokens.
    
    Args:
        all_predictions: Dictionary from activate_hierarchy()
        fallback_levels: Levels to try in order (default: [3, 2, 1, 0])
        verbose: Print which level was used
        
    Returns:
        Tuple of (predictions list, level used)
    """
    for level in fallback_levels:
        predictions = all_predictions.get(level, {}).get('predictions', [])
        
        if len(predictions) > 0:
            # Filter predictions with non-empty 'future' field
            valid_predictions = [
                pred for pred in predictions
                if pred.get('future') and len(pred.get('future', [])) > 0
            ]
            
            if len(valid_predictions) > 0:
                if verbose:
                    print(f"\n✓ Using node{level} predictions ({len(valid_predictions)} patterns with usable future data)")
                    if len(valid_predictions) < len(predictions):
                        filtered = len(predictions) - len(valid_predictions)
                        print(f"  (Filtered out {filtered} predictions with empty 'future' field)")
                return valid_predictions, level
            elif verbose:
                print(f"\n⚠ node{level} has {len(predictions)} predictions but all have empty 'future' fields")
    
    # No predictions at any level with usable future data
    if verbose:
        print("\n⚠ No predictions with non-empty 'future' field at any level")
        print("   (Input may be novel, or predictions don't contain future data)")
    
    return [], -1

print("✓ get_prediction_ensemble() function defined")

✓ get_prediction_ensemble() function defined


## Function 3: Recursive Pattern Unraveling

**Purpose**: Recursively unravel a pattern from high-level → tokens (top-down).

**How it works**:
- **Base case** (level 0): Pattern contains tokens → return them
- **Recursive case** (level > 0): Pattern contains child pattern names → unravel each child

**MongoDB Query**: This function shows the **exact MongoDB query** used to retrieve pattern structure:
```python
pattern_doc = db.find_one({'name': pattern_name})
pattern_data = pattern_doc['pattern_data']
```

**Pattern Data Structure**:
- `pattern_data = [["child1"], ["child2"], ["child3"]]`
- Each element is an event (list)
- For node0: events contain tokens
- For higher levels: events contain child pattern names

In [12]:
def unravel_pattern(
    pattern_name: str,
    level: int,
    nodes: List[KATOClient],
    verbose: bool = False,
    indent: int = 0
) -> List[str]:
    """
    Recursively unravel a pattern to tokens (top-down traversal).
    
    This function takes a pattern from any level (1-3) and recursively unravels it
    down to the base tokens. Each pattern represents 15 child patterns (fixed chunking),
    and at the bottom (node0), each pattern contains 15 tokens.
    
    Args:
        pattern_name: Pattern identifier (e.g., "PTRN|abc123..." or "abc123...")
        level: Which hierarchical level this pattern is from (0-3)
        nodes: List of KATOClient instances for each level
        verbose: Print unraveling steps
        indent: Indentation level for verbose output
    
    Returns:
        List of tokens that the pattern expands to
    
    Example:
        >>> tokens = unravel_pattern("PTRN|book_xyz", level=3, 
        ...                          nodes=nodes)
    """
    prefix = "  " * indent
    
    # Clean the pattern name (remove PTRN| prefix if present)
    clean_name = strip_pattern_prefix(pattern_name)
    
    if verbose:
        print(f"{prefix}[Level {level}] Unraveling: {clean_name[:16]}...")
    
    # Query pattern from KATO API
    result = nodes[level].get_pattern(clean_name)
    inner = result.get('pattern', {})
    pattern_ch = inner.get('pattern') if inner.get('status') == 'okay' else None
    
    if pattern_ch is None:
        if verbose:
            print(f"{prefix}  ⚠️ Pattern not found, returning empty")
        return []
    
    # For node0 patterns, return tokens directly
    if level == 0:
        # Extract tokens from the observations
        tokens = []
        observations = pattern_ch.get('observations', [])
        for obs in observations:
            token_list = obs.get('strings', [])
            tokens.extend(token_list)
        
        if verbose:
            print(f"{prefix}  → {len(tokens)} tokens")
        
        return tokens
    
    # For higher levels, recurse on each child pattern
    all_tokens = []
    observations = pattern_ch.get('observations', [])
    
    if verbose:
        print(f"{prefix}  Found {len(observations)} child patterns")
    
    for obs in observations:
        strings = obs.get('strings', [])
        
        # Child patterns may also have 'PTRN|' prefix - handle properly
        for child_pattern in strings:
            if verbose:
                clean_child = strip_pattern_prefix(child_pattern)
                print(f"{prefix}  Child: {clean_child[:16]}...")
            
            # Recursively unravel the child pattern
            child_tokens = unravel_pattern(
                child_pattern,
                level - 1,
                nodes=nodes,
                verbose=verbose,
                indent=indent + 1
            )
            all_tokens.extend(child_tokens)
    
    if verbose:
        print(f"{prefix}  Total: {len(all_tokens)} tokens")
    
    return all_tokens

In [13]:
def unravel_future_list(
    future_list: List,
    future_level: int,
    nodes: List[KATOClient],
    verbose: bool = False
) -> List[str]:
    """
    Unravel a list of future patterns to tokens.
    
    Unlike the complex unravel_prediction_with_context that was causing topic mixing,
    this function ONLY unravels the direct 'future' field from the selected prediction.
    It does NOT traverse to other predictions or collect all futures recursively.
    
    Args:
        future_list: List of future patterns from prediction['future']
        future_level: Level at which these future patterns exist (0-3)
        nodes: List of KATOClient instances for each level
        verbose: Print detailed unraveling steps
    
    Returns:
        List of tokens from unraveling the future patterns
    """
    if verbose:
        print(f"  Unraveling {len(future_list)} future events from level {future_level}")
    
    all_tokens = []
    
    # Base case: If future_level is 0 or negative, items are already tokens
    if future_level <= 0:
        # These are already tokens, just extract them
        for event in future_list:
            if isinstance(event, list) and len(event) > 0:
                all_tokens.append(event[0])
            elif isinstance(event, str):
                all_tokens.append(event)
        
        if verbose:
            print(f"    → Extracted {len(all_tokens)} tokens directly")
        
        return all_tokens
    
    # Recursive case: future contains pattern names, unravel via KATO API
    for event in future_list:
        # Extract pattern name from event
        # Future events can be: ["pattern_name"] or "pattern_name"
        if isinstance(event, list) and len(event) > 0:
            pattern_name = event[0]
        elif isinstance(event, str):
            pattern_name = event
        else:
            continue
        
        if verbose:
            clean_name = strip_pattern_prefix(pattern_name)
            print(f"  Unraveling future pattern: {clean_name[:16]}...")
        
        # Unravel this pattern
        tokens = unravel_pattern(
            pattern_name,
            level=future_level,
            nodes=nodes,
            verbose=verbose,
            indent=2 if verbose else 0
        )
        all_tokens.extend(tokens)
    
    if verbose:
        print(f"  Total future tokens: {len(all_tokens)}")
    
    return all_tokens

In [14]:
def unravel_pattern(
    pattern_name: str,
    level: int,
    nodes: List[KATOClient],
    verbose: bool = False,
    indent: int = 0
) -> List[str]:
    """
    Recursively unravel a pattern to tokens using KATO's API (top-down traversal).
    
    This function uses KATO's get_pattern() API to retrieve pattern structure, then:
    - If level 0 (node0): Return the tokens directly
    - If level > 0: Recursively unravel each child pattern
    
    Args:
        pattern_name: Pattern identifier (e.g., "PTRN|abc123..." or "abc123...")
        level: Which hierarchical level this pattern is from (0-3)
        nodes: List of KATOClient instances
        verbose: Print unraveling steps
        indent: Indentation level for verbose output
        
    Returns:
        List of token strings
        
    Example:
        >>> tokens = unravel_pattern("PTRN|book_xyz", level=3, nodes=nodes)
    """
    prefix = "  " * indent
    
    if verbose:
        print(f"{prefix}Unraveling node{level}: {pattern_name[:60]}...")
    
    # Strip 'PTRN|' prefix if present
    clean_name = strip_pattern_prefix(pattern_name)
    
    # Get pattern via KATO API (not direct database!)
    result = nodes[level].get_pattern(clean_name)
    inner = result.get('pattern', {})
    pattern_ch = inner.get('pattern') if inner.get('status') == 'okay' else None
    
    if not pattern_ch:
        if verbose:
            print(f"{prefix}  ⚠ Pattern not found via KATO API")
            print(f"{prefix}    Searched for: {clean_name}")
        return []
    
    pattern_data = pattern_ch.get('pattern_data', [])
    
    if verbose:
        print(f"{prefix}  Pattern has {len(pattern_data)} events")
    
    # Base case: node0 patterns contain actual tokens
    if level == 0:
        # Extract tokens from events
        # pattern_data = [["The"], [" cat"], [" sat"]]
        tokens = [event[0] for event in pattern_data if len(event) > 0]
        
        if verbose:
            decoded = tokenizer.decode_tokens(tokens)
            print(f"{prefix}  → Tokens: {tokens[:10]}... → '{decoded[:50]}...'")
        
        return tokens
    
    # Recursive case: higher-level patterns contain child pattern names
    else:
        # Extract child pattern names from events and strip prefix
        child_patterns = [
            strip_pattern_prefix(event[0]) 
            for event in pattern_data 
            if len(event) > 0
        ]
        
        if verbose:
            print(f"{prefix}  → Has {len(child_patterns)} children at node{level-1}")
        
        # Recursively unravel each child pattern
        all_tokens = []
        for child_pattern in child_patterns:
            # Recursive call: unravel child at level-1
            child_tokens = unravel_pattern(
                child_pattern,
                level - 1,
                nodes=nodes,
                verbose=verbose,
                indent=indent + 1
            )
            all_tokens.extend(child_tokens)
        
        if verbose:
            print(f"{prefix}  ✓ Unraveled to {len(all_tokens)} total tokens")
        
        return all_tokens

print("✓ unravel_pattern() function defined (using KATO API)")

✓ unravel_pattern() function defined (using KATO API)


In [15]:
def extract_tokens_from_present(present_events, level=0, nodes=None, verbose=False):
    """
    Extract tokens from pred['present'] field (KATO event format).
    
    The 'present' field structure varies by hierarchical level:
    - At node0: Contains actual tokens [['The'], ['cat'], ['sat']]
    - At node1+: Contains pattern names [['PTRN|abc123'], ['PTRN|def456']]
    
    For higher levels, pattern names must be unraveled recursively to get tokens.
    
    This avoids using pred['name'] which returns the full stored pattern
    (including future tokens from training time), causing repetition.
    
    Args:
        present_events: List of KATO events from pred['present']
        level: Hierarchical level (0 = node0, 1 = node1, etc.)
        nodes: List of KATOClient instances (required for level > 0)
        verbose: Print unraveling details
    
    Returns:
        List of token strings
    """
    if not present_events:
        return []
    
    if level == 0:
        # node0: Extract tokens directly (present contains actual tokens)
        tokens = []
        for event in present_events:
            # Each event is a list of strings (could have anomalies)
            # For token-level events, typically just one string per event
            if event and len(event) > 0:
                tokens.append(event[0])  # Take first string from event
        return tokens
    else:
        # node1+: Unravel pattern names recursively (present contains pattern names)
        # CRITICAL: pred['present'] at level N contains level N-1 patterns!
        # E.g., node2 prediction's present contains node1 pattern names
        # We unravel at level-1 because that's where the patterns actually exist
        if nodes is None:
            raise ValueError("nodes parameter required for hierarchical unraveling (level > 0)")
        
        all_tokens = []
        for event in present_events:
            if event and len(event) > 0:
                pattern_name = event[0]
                
                # Strip PTRN| prefix if present
                if pattern_name.startswith('PTRN|'):
                    pattern_name = pattern_name[5:]  # Remove 'PTRN|' prefix
                
                if verbose:
                    print(f"    Unraveling present pattern: {pattern_name[:16]}... (from level {level-1})")
                
                # Recursively unravel this pattern to tokens
                # Pattern is from level-1 (the child level that was observed)
                # level-1 is safe because we only reach this branch if level > 0
                tokens = unravel_pattern(
                    pattern_name,
                    level=level-1,
                    nodes=nodes,
                    verbose=verbose,
                    indent=2 if verbose else 0
                )
                
                if tokens:
                    all_tokens.extend(tokens)
                elif verbose:
                    print(f"      Warning: Failed to unravel pattern {pattern_name[:16]}...")
        
        return all_tokens


def generate_text(
    input_text: str,
    max_predictions: int = 5,
    verbose: bool = True,
    verbose_unravel: bool = False,
    recall_threshold_overrides: Dict[str, Dict[str, Any]] = None,
    auto_adjust_recall: bool = False
) -> List[Dict[str, Any]]:
    """
    Generate text continuations using hierarchical KATO system.
    
    Complete pipeline:
    1. Activate hierarchy with input text (bottom-up)
    2. Get prediction ensemble from highest available level with non-empty 'future'
    3. Extract BOTH 'present' (matched pattern) and 'future' (predicted next)
    4. Unravel both present and future predictions to tokens (top-down)
    5. Combine: present tokens + future tokens
    6. Decode combined tokens to text
    
    Args:
        input_text: User input to condition generation
        max_predictions: Maximum number of predictions to generate
        verbose: Print pipeline steps
        verbose_unravel: Print detailed unraveling steps
        recall_threshold_overrides: Optional per-node recall_threshold overrides
            Format: {'node0': {'recall_threshold': 0.3}, 'node1': {'recall_threshold': 0.6}, ...}
        auto_adjust_recall: If True, automatically lower recall_threshold if no predictions found
        
    Returns:
        List of dicts with keys:
            - 'text': Generated text string
            - 'potential': Potential metric (used for ranking)
            - 'confidence': Confidence metric
            - 'similarity': Similarity metric
            - 'evidence': Evidence metric
            - 'bayesian_posterior': Bayesian posterior probability
            - 'bayesian_prior': Bayesian prior probability
            - 'bayesian_likelihood': Bayesian likelihood
            - 'predictive_information': Predictive information metric
            - 'snr': Signal-to-noise ratio
            - 'entropy': Pattern entropy
            - 'normalized_entropy': Normalized entropy
            - 'global_normalized_entropy': Globally normalized entropy
            - 'frequency': Pattern occurrence frequency
            - 'pattern_probability': Pattern probability
            - 'weighted_strength': Weighted strength metric
            - 'fragmentation': Pattern fragmentation
            - 'itfdf_similarity': ITFDF similarity score
        
    Example:
        >>> # Generate with custom recall thresholds
        >>> results = generate_text(
        ...     "The cat sat",
        ...     recall_threshold_overrides={
        ...         'node0': {'recall_threshold': 0.3},
        ...         'node1': {'recall_threshold': 0.6}
        ...     }
        ... )
    """
    print(f"\n{'#'*80}")
    print(f"# HIERARCHICAL TEXT GENERATION (PRESENT + FUTURE)")
    print(f"{'#'*80}")
    print(f"\nInput: {input_text}")
    
    # Step 1: Activate hierarchy (bottom-up)
    all_predictions = activate_hierarchy(
        input_text,
        verbose=verbose,
        recall_threshold_overrides=recall_threshold_overrides
    )
    
    # Step 2: Get prediction ensemble with non-empty 'future' (with fallback)
    predictions, used_level = get_prediction_ensemble(
        all_predictions,
        verbose=verbose
    )
    
    # Auto-adjust recall_threshold if no predictions found
    if not predictions and auto_adjust_recall:
        print(f"\n{'='*80}")
        print("AUTO-ADJUSTING RECALL_THRESHOLD")
        print(f"{'='*80}")
        print("⚠ No predictions with usable 'future' data.")
        print("Trying progressively lower thresholds...\n")
        
        # Try decreasing thresholds
        for threshold in [0.5, 0.4, 0.3, 0.2, 0.1]:
            print(f"Trying recall_threshold={threshold}...")
            
            # Create override dict for all nodes
            override = {
                f'node{i}': {'recall_threshold': threshold}
                for i in range(4)
            }
            
            # Clear all STMs before re-activating
            # IMPORTANT: Must clear after changing recall_threshold!
            for node in nodes:
                node.clear_stm()
            
            # Re-activate hierarchy with new threshold
            all_predictions = activate_hierarchy(
                input_text,
                verbose=False,  # Suppress verbose for retry attempts
                recall_threshold_overrides=override
            )
            
            # Check if we got predictions with usable future
            predictions, used_level = get_prediction_ensemble(
                all_predictions,
                verbose=False
            )
            
            if predictions:
                print(f"✓ Found {len(predictions)} predictions with usable 'future' at recall_threshold={threshold}\n")
                print(f"{'='*80}\n")
                if verbose:
                    print(f"✓ Using node{used_level} predictions ({len(predictions)} patterns)")
                break
        
        if not predictions:
            print("⚠ No predictions with usable 'future' found even with minimum recall_threshold (0.1)")
            print(f"{'='*80}\n")
    
    if not predictions:
        print("\n⚠ No predictions with usable 'future' field available.")
        print("   (Input may be novel, or predictions don't contain future data)")
        return []
    
    # Limit to max_predictions
    predictions = predictions[:max_predictions]
    
    # Step 3: Unravel BOTH 'present' (matched pattern) and 'future' fields
    print(f"\n{'='*80}")
    print("TOP-DOWN UNRAVELING (Present + Future)")
    print(f"{'='*80}")
    print(f"\n** KEY: Combining pred['present'] (matched tokens) + pred['future'] (predicted next)")
    print(f"** pred['present'] contains exact matched tokens, pred['future'] contains predicted next")
    
    results = []
    
    for i, pred in enumerate(predictions):
        present_events = pred.get('present', [])  # Matched sequence (KATO events)
        future_list = pred.get('future', [])  # Predicted next patterns/tokens
        
        # Extract ALL available metrics from the prediction (17 total)
        metrics = {
            # Core Quality Metrics
            'potential': pred.get('potential', 0.0),
            'confidence': pred.get('confidence', 0.0),
            'similarity': pred.get('similarity', 0.0),
            'evidence': pred.get('evidence', 0.0),
            # Bayesian Metrics
            'bayesian_posterior': pred.get('bayesian_posterior', 0.0),
            'bayesian_prior': pred.get('bayesian_prior', 0.0),
            'bayesian_likelihood': pred.get('bayesian_likelihood', 0.0),
            # Information Theory Metrics
            'predictive_information': pred.get('predictive_information', 0.0),
            'snr': pred.get('snr', 0.0),
            'entropy': pred.get('entropy', 0.0),
            'normalized_entropy': pred.get('normalized_entropy', 0.0),
            'global_normalized_entropy': pred.get('global_normalized_entropy', 0.0),
            # Pattern Strength Metrics
            'frequency': pred.get('frequency', 0),
            'pattern_probability': pred.get('pattern_probability', 0.0),
            'weighted_strength': pred.get('weighted_strength', 0.0),
            'fragmentation': pred.get('fragmentation', 0.0),
            'itfdf_similarity': pred.get('itfdf_similarity', 0.0),
            'tfidf_score': pred.get('tfidf_score', 0.0),
        }
        
        print(f"\nPrediction {i+1}/{len(predictions)}:")
        print(f"  Matched Pattern: {pred['name'][:60]}...")
        print(f"  Level: node{used_level}")
        print(f"  Present Events: {len(present_events)} tokens")
        print(f"  Future Events: {len(future_list)} events")
        
        # Display all metrics in grouped format
        print(f"\n  Prediction Metrics:")
        print(f"    --- Core Quality ---")
        print(f"    Potential:                  {metrics['potential']:.4f}")
        print(f"    Confidence:                 {metrics['confidence']:.4f}")
        print(f"    Similarity:                 {metrics['similarity']:.4f}")
        print(f"    Evidence:                   {metrics['evidence']:.4f}")
        print(f"    --- Bayesian ---")
        print(f"    Bayesian Posterior:         {metrics['bayesian_posterior']:.4f}")
        print(f"    Bayesian Prior:             {metrics['bayesian_prior']:.4f}")
        print(f"    Bayesian Likelihood:        {metrics['bayesian_likelihood']:.4f}")
        print(f"    --- Information Theory ---")
        print(f"    Predictive Information:     {metrics['predictive_information']:.4f}")
        print(f"    SNR:                        {metrics['snr']:.4f}")
        print(f"    Entropy:                    {metrics['entropy']:.4f}")
        print(f"    Normalized Entropy:         {metrics['normalized_entropy']:.4f}")
        print(f"    Global Normalized Entropy:  {metrics['global_normalized_entropy']:.4f}")
        print(f"    --- Pattern Strength ---")
        print(f"    Frequency:                  {metrics['frequency']}")
        print(f"    Pattern Probability:        {metrics['pattern_probability']:.4f}")
        print(f"    Weighted Strength:          {metrics['weighted_strength']:.4f}")
        print(f"    Fragmentation:              {metrics['fragmentation']:.4f}")
        print(f"    ITFDF Similarity:           {metrics['itfdf_similarity']:.4f}")
        print(f"    TF-IDF Score:               {metrics['tfidf_score']:.4f}")
        
        # First, extract the PRESENT tokens (unravel if from higher level)
        if verbose_unravel:
            print(f"\n  Extracting PRESENT tokens from pred['present'] field...")
        
        present_tokens = extract_tokens_from_present(
            present_events,
            level=used_level,
            nodes=nodes,
            verbose=verbose_unravel
        )
        
        if not present_tokens:
            print(f"  ⚠ Failed to unravel present pattern")
            # Even if present fails, try to get future
        
        # Then, unravel the FUTURE (predicted next patterns/tokens)
        if not future_list:
            print(f"  ⚠ Empty 'future' field")
            if not present_tokens:
                continue  # Skip if both present and future are empty
            # If we have present but no future, just use present
            tokens = present_tokens
            future_tokens = []
        else:
            # Calculate future level
            future_level = used_level - 1 if used_level > 0 else -1
            
            if verbose_unravel:
                print(f"\n  Unraveling FUTURE (predicted next) from level {future_level}...")
            
            future_tokens = unravel_future_list(
                future_list,
                future_level=future_level,
                nodes=nodes,
                verbose=verbose_unravel
            )
            
            if not future_tokens:
                print(f"  ⚠ Failed to unravel future")
                # If future fails but we have present, use just present
                if present_tokens:
                    tokens = present_tokens
                else:
                    continue
            else:
                # Combine present + future tokens
                tokens = present_tokens + future_tokens
        
        # Decode combined tokens to text
        generated_text = tokenizer.decode_tokens(tokens)

        _present = tokenizer.decode_tokens(present_tokens) if present_tokens else ""
        _future = tokenizer.decode_tokens(future_tokens) if future_tokens else ""
        
        print(f"\n  ✓ Generated:")
        print(f"     Present tokens: {len(present_tokens)}")
        print(f"     Future tokens: {len(future_tokens)}")
        print(f"     Total tokens: {len(tokens)}")
        print(f"=="*20)
        print(f"  Present: {_present}")
        print(f"--"*20)
        print(f"  Future: {_future}")
        print(f"=="*20)
        print(f"  Text preview: {generated_text}")
        
        # Store result with all metrics
        result_dict = {
            'text': generated_text,
            **metrics  # Unpack all metrics into the result dict
        }
        results.append(result_dict)
    
    # Sort by potential (primary ranking metric), fallback to confidence
    results.sort(key=lambda x: x['potential'] if x['potential'] > 0 else x['confidence'], reverse=True)
    
    print(f"\n{'='*80}")
    print(f"GENERATION COMPLETE: {len(results)} results")
    print(f"{'='*80}")
    
    return results

print("✓ generate_text() function defined (with PRESENT + FUTURE + ALL 17 METRICS)")

✓ generate_text() function defined (with PRESENT + FUTURE + ALL 17 METRICS)


### Example 1: Simple Input

In [16]:
# Simple short input
input_text = "The cat sat on the "

results = generate_text(
    input_text=input_text,
    # max_predictions=100,
    verbose=True,
    verbose_unravel=False  # Set True to see detailed unraveling
)

# Display results
print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"  Metrics:")
    print(f"    --- Core Quality ---")
    print(f"    Potential:                  {result['potential']:.4f}")
    print(f"    Confidence:                 {result['confidence']:.4f}")
    print(f"    Similarity:                 {result['similarity']:.4f}")
    print(f"    Evidence:                   {result['evidence']:.4f}")
    print(f"    --- Bayesian ---")
    print(f"    Bayesian Posterior:         {result['bayesian_posterior']:.4f}")
    print(f"    Bayesian Prior:             {result['bayesian_prior']:.4f}")
    print(f"    Bayesian Likelihood:        {result['bayesian_likelihood']:.4f}")
    print(f"    --- Information Theory ---")
    print(f"    Predictive Information:     {result['predictive_information']:.4f}")
    print(f"    SNR:                        {result['snr']:.4f}")
    print(f"    Entropy:                    {result['entropy']:.4f}")
    print(f"    Normalized Entropy:         {result['normalized_entropy']:.4f}")
    print(f"    Global Normalized Entropy:  {result['global_normalized_entropy']:.4f}")
    print(f"    --- Pattern Strength ---")
    print(f"    Frequency:                  {result['frequency']}")
    print(f"    Pattern Probability:        {result['pattern_probability']:.4f}")
    print(f"    Weighted Strength:          {result['weighted_strength']:.4f}")
    print(f"    Fragmentation:              {result['fragmentation']:.4f}")
    print(f"    ITFDF Similarity:           {result['itfdf_similarity']:.4f}")
    print(f"    TF-IDF Score:               {result['tfidf_score']:.4f}")
    print(f"  Text: {result['text']}")
    print()


################################################################################
# HIERARCHICAL TEXT GENERATION (PRESENT + FUTURE)
################################################################################

Input: The cat sat on the 

BOTTOM-UP ACTIVATION (Chunk-by-Chunk Cascading)
Input: The cat sat on the 
Config: chunk_sizes=[8, 8, 8, 8], max_pred=[10, 10, 10, 10]

Tokens (6): ['The', 'Ġcat', 'Ġsat', 'Ġon', 'Ġthe', 'Ġ']

Chunks (1) with chunk_size=8:
  Chunk 0: ['The', 'Ġcat', 'Ġsat', 'Ġon', 'Ġthe', 'Ġ']

--- INITIALIZING: Clearing all nodes' STM ---
✓ Cleared node0 STM
✓ Cleared node1 STM
✓ Cleared node2 STM
✓ Cleared node3 STM

CHUNK 1/1: ['The', 'Ġcat', 'Ġsat', 'Ġon', 'Ġthe', 'Ġ']
✓ Cleared node0 STM for new chunk

--- NODE0 ---
✓ Observed 6 tokens
  node0 STM: 6 events
     STM: [['The'], ['Ġcat'], ['Ġsat'], ['Ġon'], ['Ġthe'], ['Ġ']]
✓ Got 10 predictions
  Sample predictions:
    1. e3ad92dd06caca4e0ef3156504e8400d64187c85... (conf: 1.000)
    2. c0a6516eb75369b71dfd7b99e

### Example 2: Longer Input (More Context)

In [17]:
# Longer input with more context
input_text = "Machine learning is a field of artificial intelligence that"

results = generate_text(
    input_text=input_text,
    # max_predictions=3,
    verbose=True,
    verbose_unravel=False
)

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"  Metrics:")
    print(f"    --- Core Quality ---")
    print(f"    Potential:                  {result['potential']:.4f}")
    print(f"    Confidence:                 {result['confidence']:.4f}")
    print(f"    Similarity:                 {result['similarity']:.4f}")
    print(f"    Evidence:                   {result['evidence']:.4f}")
    print(f"    --- Bayesian ---")
    print(f"    Bayesian Posterior:         {result['bayesian_posterior']:.4f}")
    print(f"    Bayesian Prior:             {result['bayesian_prior']:.4f}")
    print(f"    Bayesian Likelihood:        {result['bayesian_likelihood']:.4f}")
    print(f"    --- Information Theory ---")
    print(f"    Predictive Information:     {result['predictive_information']:.4f}")
    print(f"    SNR:                        {result['snr']:.4f}")
    print(f"    Entropy:                    {result['entropy']:.4f}")
    print(f"    Normalized Entropy:         {result['normalized_entropy']:.4f}")
    print(f"    Global Normalized Entropy:  {result['global_normalized_entropy']:.4f}")
    print(f"    --- Pattern Strength ---")
    print(f"    Frequency:                  {result['frequency']}")
    print(f"    Pattern Probability:        {result['pattern_probability']:.4f}")
    print(f"    Weighted Strength:          {result['weighted_strength']:.4f}")
    print(f"    Fragmentation:              {result['fragmentation']:.4f}")
    print(f"    ITFDF Similarity:           {result['itfdf_similarity']:.4f}")
    print(f"    TF-IDF Score:               {result['tfidf_score']:.4f}")
    print(f"  Text: {result['text']}")
    print()


################################################################################
# HIERARCHICAL TEXT GENERATION (PRESENT + FUTURE)
################################################################################

Input: Machine learning is a field of artificial intelligence that

BOTTOM-UP ACTIVATION (Chunk-by-Chunk Cascading)
Input: Machine learning is a field of artificial intelligence that
Config: chunk_sizes=[8, 8, 8, 8], max_pred=[10, 10, 10, 10]

Tokens (9): ['Machine', 'Ġlearning', 'Ġis', 'Ġa', 'Ġfield', 'Ġof', 'Ġartificial', 'Ġintelligence', 'Ġthat']

Chunks (2) with chunk_size=8:
  Chunk 0: ['Machine', 'Ġlearning', 'Ġis', 'Ġa', 'Ġfield', 'Ġof', 'Ġartificial', 'Ġintelligence']
  Chunk 1: ['Ġthat']

--- INITIALIZING: Clearing all nodes' STM ---
✓ Cleared node0 STM
✓ Cleared node1 STM
✓ Cleared node2 STM
✓ Cleared node3 STM

CHUNK 1/2: ['Machine', 'Ġlearning', 'Ġis', 'Ġa', 'Ġfield', 'Ġof', 'Ġartificial', 'Ġintelligence']
✓ Cleared node0 STM for new chunk

--- NODE0 ---
✓ Observe

In [21]:
node0.get_predictions()

{'predictions': [],
 'future_potentials': [{'future': [['Ġcharacteristic'],
    ['Ġp'],
    ['Ġ,'],
    ['Ġand']],
   'aggregate_potential': 0.5,
   'supporting_patterns': 1,
   'total_weighted_frequency': 0.5},
  {'future': [['Ġresearch'], ['Ġwas'], ['Ġfounded']],
   'aggregate_potential': 0.5,
   'supporting_patterns': 1,
   'total_weighted_frequency': 0.5}],
 'session_id': 'session-6e6d2f9dcb794b05898de0f2d69c3b08-1770237360667',
 'processor_id': None,
 'count': 0,
 'time': None,
 'unique_id': None}

In [19]:
node1.get_predictions()

{'predictions': [{'type': 'prototypical',
   'name': 'a134a2f225bb8d031deedeb8e88ef8d13c869f48',
   'frequency': 1,
   'emotives': {},
   'matches': ['PTRN|a6dd9b5cbba7b644cfc671af13d5d8f2f835d80f'],
   'past': [['PTRN|feb15b241126e6dd3cfe6c11d36df820707f9320']],
   'present': [['PTRN|a6dd9b5cbba7b644cfc671af13d5d8f2f835d80f']],
   'missing': [[]],
   'extras': [['PTRN|f9b44c1ec1d7e8f5d9107720072a54c37f768aea']],
   'anomalies': [],
   'potential': 2.228553390593274,
   'evidence': 0.125,
   'similarity': 0.2,
   'fragmentation': 0.0,
   'snr': 0.3333333333333333,
   'confluence': 7.901827692251944e-06,
   'predictive_information': 0.2,
   'sequence': [['PTRN|feb15b241126e6dd3cfe6c11d36df820707f9320'],
    ['PTRN|a6dd9b5cbba7b644cfc671af13d5d8f2f835d80f'],
    ['PTRN|2e86b0e051a29ea19afb5ffac8c868eab98025af'],
    ['PTRN|78f6657d7f60378ad76a2ebb2b504548da644d9d'],
    ['PTRN|641e381be6d670a8b75c2e56859d2d9a27c3d08d'],
    ['PTRN|c60364fef3ab841fa6344b885f74fe3ce019da02'],
    ['PTRN|e8

In [20]:
node2.get_predictions()

{'predictions': [{'type': 'prototypical',
   'name': '1af413a8bf69575568106c530fd59dffb25b918a',
   'frequency': 1,
   'emotives': {},
   'matches': ['PTRN|a134a2f225bb8d031deedeb8e88ef8d13c869f48'],
   'past': [['PTRN|126885a0e82cb47df4fe488b35adc98c944d49ee']],
   'present': [['PTRN|a134a2f225bb8d031deedeb8e88ef8d13c869f48']],
   'missing': [[]],
   'extras': [['PTRN|7ada9600ca3d3abd5b4fda177bd7791cb7f17ce1']],
   'anomalies': [],
   'potential': 2.4444444444444446,
   'evidence': 0.3333333333333333,
   'similarity': 0.4,
   'fragmentation': 0.0,
   'snr': 0.3333333333333333,
   'confluence': 2.740101381693775e-05,
   'predictive_information': 0.4,
   'sequence': [['PTRN|126885a0e82cb47df4fe488b35adc98c944d49ee'],
    ['PTRN|a134a2f225bb8d031deedeb8e88ef8d13c869f48'],
    ['PTRN|f9817e5c05758fc1020577c074544c09aaebbb89']],
   'future': [['PTRN|f9817e5c05758fc1020577c074544c09aaebbb89']],
   'confidence': 1.0,
   'entropy': 0.0,
   'normalized_entropy': 0.059801911435595254,
   'globa

In [18]:
node3.get_predictions()

{'predictions': [],
 'future_potentials': [],
 'session_id': 'session-6b0e7eea03cf43548464ed6166d13a7c-1770237360864',
 'processor_id': None,
 'count': 0,
 'time': None,
 'unique_id': None}

### Example 3: Custom Input (Try Your Own!)

In [22]:
# Try your own input!
input_text = "The researchers found that"  # <-- Change this

results = generate_text(
    input_text=input_text,
    # max_predictions=100,
    verbose=True,
    verbose_unravel=True
)

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"  Metrics:")
    print(f"    --- Core Quality ---")
    print(f"    Potential:                  {result['potential']:.4f}")
    print(f"    Confidence:                 {result['confidence']:.4f}")
    print(f"    Similarity:                 {result['similarity']:.4f}")
    print(f"    Evidence:                   {result['evidence']:.4f}")
    print(f"    --- Bayesian ---")
    print(f"    Bayesian Posterior:         {result['bayesian_posterior']:.4f}")
    print(f"    Bayesian Prior:             {result['bayesian_prior']:.4f}")
    print(f"    Bayesian Likelihood:        {result['bayesian_likelihood']:.4f}")
    print(f"    --- Information Theory ---")
    print(f"    Predictive Information:     {result['predictive_information']:.4f}")
    print(f"    SNR:                        {result['snr']:.4f}")
    print(f"    Entropy:                    {result['entropy']:.4f}")
    print(f"    Normalized Entropy:         {result['normalized_entropy']:.4f}")
    print(f"    Global Normalized Entropy:  {result['global_normalized_entropy']:.4f}")
    print(f"    --- Pattern Strength ---")
    print(f"    Frequency:                  {result['frequency']}")
    print(f"    Pattern Probability:        {result['pattern_probability']:.4f}")
    print(f"    Weighted Strength:          {result['weighted_strength']:.4f}")
    print(f"    Fragmentation:              {result['fragmentation']:.4f}")
    print(f"    ITFDF Similarity:           {result['itfdf_similarity']:.4f}")
    print(f"    TF-IDF Score:               {result['tfidf_score']:.4f}")
    print(f"  Text: {result['text']}")
    print()


################################################################################
# HIERARCHICAL TEXT GENERATION (PRESENT + FUTURE)
################################################################################

Input: The researchers found that

BOTTOM-UP ACTIVATION (Chunk-by-Chunk Cascading)
Input: The researchers found that
Config: chunk_sizes=[8, 8, 8, 8], max_pred=[10, 10, 10, 10]

Tokens (4): ['The', 'Ġresearchers', 'Ġfound', 'Ġthat']

Chunks (1) with chunk_size=8:
  Chunk 0: ['The', 'Ġresearchers', 'Ġfound', 'Ġthat']

--- INITIALIZING: Clearing all nodes' STM ---
✓ Cleared node0 STM
✓ Cleared node1 STM
✓ Cleared node2 STM
✓ Cleared node3 STM

CHUNK 1/1: ['The', 'Ġresearchers', 'Ġfound', 'Ġthat']
✓ Cleared node0 STM for new chunk

--- NODE0 ---
✓ Observed 4 tokens
  node0 STM: 4 events
     STM: [['The'], ['Ġresearchers'], ['Ġfound'], ['Ġthat']]
✓ Got 2 predictions
  Sample predictions:
    1. 1967e8489f2c5e298c4006662da3cbe8709bf715... (conf: 1.000)
    2. 43d30465d6248c07718f

### Example 4: Known sample

The following includes exact phrases that are known to have been learned by the agent.

First, we'll provide the STM with a little context. This won't produce robust predictions. But, next we'll take the output as predicted, add it to the STM, and request a second prediction.

Here's the first

In [15]:
node0.get_session_info()

{'session_id': 'session-805abf96df4f457c8add72cbb74fe1f4-1770227190343',
 'node_id': 'node0',
 'created_at': '2026-02-04T17:46:30.343413Z',
 'expires_at': '2026-02-04T18:47:10.327325Z',
 'ttl_seconds': 3599,
 'metadata': {},
 'session_config': {'max_pattern_length': 0,
  'persistence': 5,
  'recall_threshold': 0.3,
  'indexer_type': 'VI',
  'max_predictions': 10,
  'sort_symbols': True,
  'process_predictions': False,
  'use_token_matching': True,
  'stm_mode': 'CLEAR',
  'rank_sort_algo': 'potential',
  'filter_pipeline': ['jaccard'],
  'length_min_ratio': 0.5,
  'length_max_ratio': 2.0,
  'jaccard_threshold': 0.3,
  'jaccard_min_overlap': 2,
  'minhash_threshold': 0.7,
  'minhash_bands': 20,
  'minhash_rows': 5,
  'minhash_num_hashes': 100,
  'bloom_false_positive_rate': 0.01,
  'max_candidates_per_stage': 1000,
  'enable_filter_metrics': True}}

In [23]:
# Test with various inputs to see generation quality
# The system now combines PRESENT (matched pattern) + FUTURE (predicted continuation)

# Example inputs - uncomment the one you want to test:

# input_text = "The cat sat"
# input_text = "the most common in North American wolves"
# input_text = "Tapeworms generally cause little harm in wolves, though this depends on the number and size of the parasites, and the sensitivity of the host."

# This input finds a rich pattern about wolf parasites (830 tokens total with present+future):
input_text = " Among flukes, the most common"  # longer gets more: " Among flukes, the most common in"  

# Note: The space at the beginning and "in" at the end are important for matching the correct pattern
# This demonstrates how exact phrasing affects pattern matching

results = generate_text(
    input_text=input_text,
    verbose=True,
    verbose_unravel=True  # Set to True to see detailed unraveling
)

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"  Metrics:")
    print(f"    --- Core Quality ---")
    print(f"    Potential:                  {result['potential']:.4f}")
    print(f"    Confidence:                 {result['confidence']:.4f}")
    print(f"    Similarity:                 {result['similarity']:.4f}")
    print(f"    Evidence:                   {result['evidence']:.4f}")
    print(f"    --- Bayesian ---")
    print(f"    Bayesian Posterior:         {result['bayesian_posterior']:.4f}")
    print(f"    Bayesian Prior:             {result['bayesian_prior']:.4f}")
    print(f"    Bayesian Likelihood:        {result['bayesian_likelihood']:.4f}")
    print(f"    --- Information Theory ---")
    print(f"    Predictive Information:     {result['predictive_information']:.4f}")
    print(f"    SNR:                        {result['snr']:.4f}")
    print(f"    Entropy:                    {result['entropy']:.4f}")
    print(f"    Normalized Entropy:         {result['normalized_entropy']:.4f}")
    print(f"    Global Normalized Entropy:  {result['global_normalized_entropy']:.4f}")
    print(f"    --- Pattern Strength ---")
    print(f"    Frequency:                  {result['frequency']}")
    print(f"    Pattern Probability:        {result['pattern_probability']:.4f}")
    print(f"    Weighted Strength:          {result['weighted_strength']:.4f}")
    print(f"    Fragmentation:              {result['fragmentation']:.4f}")
    print(f"    ITFDF Similarity:           {result['itfdf_similarity']:.4f}")
    print(f"    TF-IDF Score:               {result['tfidf_score']:.4f}")
    print(f"  Text length: {len(result['text'])} chars")
    print(f"  Text: {result['text']}")
    print()


################################################################################
# HIERARCHICAL TEXT GENERATION (PRESENT + FUTURE)
################################################################################

Input:  Among flukes, the most common

BOTTOM-UP ACTIVATION (Chunk-by-Chunk Cascading)
Input:  Among flukes, the most common
Config: chunk_sizes=[8, 8, 8, 8], max_pred=[10, 10, 10, 10]

Tokens (7): ['ĠAmong', 'Ġfl', 'ukes', ',', 'Ġthe', 'Ġmost', 'Ġcommon']

Chunks (1) with chunk_size=8:
  Chunk 0: ['ĠAmong', 'Ġfl', 'ukes', ',', 'Ġthe', 'Ġmost', 'Ġcommon']

--- INITIALIZING: Clearing all nodes' STM ---
✓ Cleared node0 STM
✓ Cleared node1 STM
✓ Cleared node2 STM
✓ Cleared node3 STM

CHUNK 1/1: ['ĠAmong', 'Ġfl', 'ukes', ',', 'Ġthe', 'Ġmost', 'Ġcommon']
✓ Cleared node0 STM for new chunk

--- NODE0 ---
✓ Observed 7 tokens
  node0 STM: 7 events
     STM: [['ĠAmong'], ['Ġfl'], ['ukes'], [','], ['Ġthe'], ['Ġmost'], ['Ġcommon']]
✓ Got 1 predictions
  Sample predictions:
    1. 6850d8e

### Now, we'll add the predicted output, "in", back and get more predictions...

In [17]:
# Test with various inputs to see generation quality
# The system now combines PRESENT (matched pattern) + FUTURE (predicted continuation)

# Example inputs - uncomment the one you want to test:

# input_text = "The cat sat"
# input_text = "the most common in North American wolves"
# input_text = "Tapeworms generally cause little harm in wolves, though this depends on the number and size of the parasites, and the sensitivity of the host."

# This input finds a rich pattern about wolf parasites (830 tokens total with present+future):
input_text = " Among flukes, the most common in"  

# Note: The space at the beginning and "in" at the end are important for matching the correct pattern
# This demonstrates how exact phrasing affects pattern matching

results = generate_text(
    input_text=input_text,
    verbose=True,
    verbose_unravel=True  # Set to True to see detailed unraveling
)

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"  Metrics:")
    print(f"    --- Core Quality ---")
    print(f"    Potential:                  {result['potential']:.4f}")
    print(f"    Confidence:                 {result['confidence']:.4f}")
    print(f"    Similarity:                 {result['similarity']:.4f}")
    print(f"    Evidence:                   {result['evidence']:.4f}")
    print(f"    --- Bayesian ---")
    print(f"    Bayesian Posterior:         {result['bayesian_posterior']:.4f}")
    print(f"    Bayesian Prior:             {result['bayesian_prior']:.4f}")
    print(f"    Bayesian Likelihood:        {result['bayesian_likelihood']:.4f}")
    print(f"    --- Information Theory ---")
    print(f"    Predictive Information:     {result['predictive_information']:.4f}")
    print(f"    SNR:                        {result['snr']:.4f}")
    print(f"    Entropy:                    {result['entropy']:.4f}")
    print(f"    Normalized Entropy:         {result['normalized_entropy']:.4f}")
    print(f"    Global Normalized Entropy:  {result['global_normalized_entropy']:.4f}")
    print(f"    --- Pattern Strength ---")
    print(f"    Frequency:                  {result['frequency']}")
    print(f"    Pattern Probability:        {result['pattern_probability']:.4f}")
    print(f"    Weighted Strength:          {result['weighted_strength']:.4f}")
    print(f"    Fragmentation:              {result['fragmentation']:.4f}")
    print(f"    ITFDF Similarity:           {result['itfdf_similarity']:.4f}")
    print(f"    TF-IDF Score:               {result['tfidf_score']:.4f}")
    print(f"  Text length: {len(result['text'])} chars")
    print(f"  Text: {result['text']}")
    print()


################################################################################
# HIERARCHICAL TEXT GENERATION (PRESENT + FUTURE)
################################################################################

Input:  Among flukes, the most common in

BOTTOM-UP ACTIVATION (Chunk-by-Chunk Cascading)
Input:  Among flukes, the most common in
Config: chunk_sizes=[8, 8, 8, 8], max_pred=[10, 10, 10, 10]

Tokens (8): ['ĠAmong', 'Ġfl', 'ukes', ',', 'Ġthe', 'Ġmost', 'Ġcommon', 'Ġin']

Chunks (1) with chunk_size=8:
  Chunk 0: ['ĠAmong', 'Ġfl', 'ukes', ',', 'Ġthe', 'Ġmost', 'Ġcommon', 'Ġin']

--- INITIALIZING: Clearing all nodes' STM ---
✓ Cleared node0 STM
✓ Cleared node1 STM
✓ Cleared node2 STM
✓ Cleared node3 STM

CHUNK 1/1: ['ĠAmong', 'Ġfl', 'ukes', ',', 'Ġthe', 'Ġmost', 'Ġcommon', 'Ġin']
✓ Cleared node0 STM for new chunk

--- NODE0 ---
✓ Observed 8 tokens
  node0 STM: 8 events
     STM: [['ĠAmong'], ['Ġfl'], ['ukes'], [','], ['Ġthe'], ['Ġmost'], ['Ġcommon'], ['Ġin']]
✓ Got 10 predictions

### Example 4b: Adjusting Prediction Ensemble Size

Control how many predictions flow between levels using `max_predictions`.

**Impact of ensemble size**:
- **Larger ensembles** (20-50): More context sent to next level → better quality, but slower and potentially noisy
- **Smaller ensembles** (3-5): Less context → faster and cleaner, but may miss important patterns
- **Recommended**: 5-15 for balanced quality and speed

**Use cases**:
- Speed-critical applications: Use small ensembles
- Quality-critical applications: Use large ensembles
- Novel inputs: Use larger ensembles to catch weak matches

In [18]:
# Compare different ensemble sizes using chunk_sizes and max_predictions parameters
input_text = "The cat sat on the mat"

# Test 1: Small ensembles (faster, cleaner)
print("=" * 80)
print("TEST 1: Small Ensembles (max_predictions=[3, 3, 3, 3])")
print("=" * 80)

results_small = activate_hierarchy(
    input_text=input_text,
    verbose=False,
    chunk_sizes=CHUNK_SIZES,      # Use global chunk_sizes
    max_predictions=[3, 3, 3, 3]  # Small ensembles
)

# Get predictions from highest level
preds_small, level_small = get_prediction_ensemble(results_small, verbose=False)

print(f"\nResults: {len(preds_small)} predictions from node{level_small}")
for i, pred in enumerate(preds_small[:2]):
    print(f"  {i+1}. {pred['name'][:60]}... (conf: {pred.get('confidence', 0):.3f})")

# Test 2: Large ensembles (slower, more context)
print("\n" + "=" * 80)
print("TEST 2: Large Ensembles (max_predictions=[20, 15, 10, 5])")
print("=" * 80)

results_large = activate_hierarchy(
    input_text=input_text,
    verbose=False,
    chunk_sizes=CHUNK_SIZES,            # Use global chunk_sizes
    max_predictions=[20, 15, 10, 5]     # Large ensembles
)

preds_large, level_large = get_prediction_ensemble(results_large, verbose=False)

print(f"\nResults: {len(preds_large)} predictions from node{level_large}")
for i, pred in enumerate(preds_large[:2]):
    print(f"  {i+1}. {pred['name'][:60]}... (conf: {pred.get('confidence', 0):.3f})")

print("\n" + "=" * 80)
print("COMPARISON")
print("=" * 80)
print(f"Small ensembles: {len(preds_small)} predictions from node{level_small}")
print(f"Large ensembles: {len(preds_large)} predictions from node{level_large}")
print("\nLarger ensembles provide more context to higher levels.")
print("This can lead to different prediction quantities and levels activated.")

TEST 1: Small Ensembles (max_predictions=[3, 3, 3, 3])

Results: 1 predictions from node2
  1. fd46712761f6833c53015b2fe771bb56c2f9d58e... (conf: 1.000)

TEST 2: Large Ensembles (max_predictions=[20, 15, 10, 5])

Results: 1 predictions from node2
  1. fd46712761f6833c53015b2fe771bb56c2f9d58e... (conf: 1.000)

COMPARISON
Small ensembles: 1 predictions from node2
Large ensembles: 1 predictions from node2

Larger ensembles provide more context to higher levels.
This can lead to different prediction quantities and levels activated.


### Example 5: Auto-Adjust recall_threshold (Cold Start Handling)

If input is very novel (not seen during training), predictions may be empty with default recall_threshold.

**auto_adjust_recall=True** will automatically try progressively lower thresholds (0.5, 0.4, 0.3, 0.2, 0.1) until predictions are found.

This is useful for:
- Novel inputs not in training data
- Cold start scenarios
- Graceful degradation (lower quality predictions rather than nothing)

In [ ]:
# If input is very novel (not seen during training), predictions may be empty
# auto_adjust_recall=True will automatically try lower thresholds

input_text = "Quantum entanglement demonstrates"  # Novel input

results = generate_text(
    input_text=input_text,
    # max_predictions=3,
    auto_adjust_recall=True,  # Enable auto-adjustment
    verbose=True
)

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

if results:
    for i, result in enumerate(results):
        print(f"\nResult {i+1}:")
        print(f"  Metrics:")
        print(f"    --- Core Quality ---")
        print(f"    Potential:                  {result['potential']:.4f}")
        print(f"    Confidence:                 {result['confidence']:.4f}")
        print(f"    Similarity:                 {result['similarity']:.4f}")
        print(f"    Evidence:                   {result['evidence']:.4f}")
        print(f"    --- Bayesian ---")
        print(f"    Bayesian Posterior:         {result['bayesian_posterior']:.4f}")
        print(f"    Bayesian Prior:             {result['bayesian_prior']:.4f}")
        print(f"    Bayesian Likelihood:        {result['bayesian_likelihood']:.4f}")
        print(f"    --- Information Theory ---")
        print(f"    Predictive Information:     {result['predictive_information']:.4f}")
        print(f"    SNR:                        {result['snr']:.4f}")
        print(f"    Entropy:                    {result['entropy']:.4f}")
        print(f"    Normalized Entropy:         {result['normalized_entropy']:.4f}")
        print(f"    Global Normalized Entropy:  {result['global_normalized_entropy']:.4f}")
        print(f"    --- Pattern Strength ---")
        print(f"    Frequency:                  {result['frequency']}")
        print(f"    Pattern Probability:        {result['pattern_probability']:.4f}")
        print(f"    Weighted Strength:          {result['weighted_strength']:.4f}")
        print(f"    Fragmentation:              {result['fragmentation']:.4f}")
        print(f"    ITFDF Similarity:           {result['itfdf_similarity']:.4f}")
        print(f"    TF-IDF Score:               {result['tfidf_score']:.4f}")
        print(f"  Text: {result['text'][:300]}...")
        print()
else:
    print("\n⚠ No predictions found. Input is completely novel to the system.")

## Debugging & Inspection

Useful cells for inspecting the system state and understanding what's happening.

### Inspect Specific Pattern

In [ ]:
# Inspect a specific pattern using KATO API
# Replace with an actual pattern name from your predictions
# Pattern name can be with or without 'PTRN|' prefix

pattern_name = "92aefd2182f65233aaa2b975b0cc292f967b745c"
level = 0

# Strip prefix before querying (handles both formats)
clean_name = strip_pattern_prefix(pattern_name)

print(f"Inspecting pattern: {pattern_name}")
print(f"Queried as: {clean_name}")
print(f"Level: node{level}\n")

# ===== Query via KATO API =====
print("="*80)
print("Pattern Query via KATO API:")
print("="*80)

# Get pattern data via KATO API
result = nodes[level].get_pattern(clean_name)
inner = result.get('pattern', {})

if inner.get('status') == 'okay':
    pattern_data = inner.get('pattern', {})
    metadata = inner.get('metadata', {})
    
    print(f"✓ Pattern found")
    print(f"  Frequency: {metadata.get('frequency', 0)}")
    
    observations = pattern_data.get('observations', [])
    print(f"  Observations: {len(observations)} events")
    
    print(f"\n  Pattern data (first 10 events):")
    for i, obs in enumerate(observations[:10]):
        print(f"    Event {i}: {obs}")
    
    if level == 0:
        # Decode tokens
        tokens = []
        for obs in observations:
            token_list = obs.get('strings', [])
            tokens.extend(token_list)
        print(f"\n  Decoded tokens ({len(tokens)} total):")
        print(f"  {' '.join(tokens)}")
    else:
        # Show child patterns
        print(f"\n  Child patterns (first 10):")
        for i, obs in enumerate(observations[:10]):
            child_patterns = obs.get('strings', [])
            for pattern in child_patterns:
                clean_child = strip_pattern_prefix(pattern)
                print(f"    {i}: {clean_child[:32]}...")
else:
    print(f"✗ Pattern not found or error: {inner.get('status', 'unknown')}")
    if 'error' in inner:
        print(f"  Error: {inner['error']}")

# ===== KATO API Response Structure =====
print("\n" + "="*80)
print("KATO API Response Structure:")
print("="*80)
print(f"Full result keys: {list(result.keys())}")
if 'pattern' in result:
    print(f"Inner pattern keys: {list(inner.keys())}")
    if 'pattern' in inner:
        print(f"Pattern data keys: {list(pattern_data.keys())}")
    if 'metadata' in inner:
        print(f"Metadata keys: {list(metadata.keys())}")

### Test Pattern Unraveling Directly

In [ ]:
# Test unraveling a specific pattern with verbose output
# This is useful for understanding the recursive unraveling process
# Pattern name can be with or without 'PTRN|' prefix

pattern_name = "PTRN|04f57fef3b3f9b0f33b9d04987b16f56755d8c63"  # <-- Put a pattern name from a higher level here
level = 3  # Which level (1, 2, or 3 to see recursion)

# pattern_name = "PTRN|811d57463ca70786022e901064c3722794296a24"
# level = 2

# pattern_name = "PTRN|d1a599707a223c611f6547a56077af442f9e1e22"
# level = 1

pattern_name = "PTRN|6850d8ef6abf023e778693c4d5d9986db464e5cd"
level = 0

print(f"Unraveling pattern from node{level}:")
print(f"Input: {pattern_name}")
print(f"Clean name: {strip_pattern_prefix(pattern_name)}")

tokens = unravel_pattern(
    pattern_name,  # Can include 'PTRN|' prefix - will be stripped automatically
    level=level,
    nodes=nodes,
    verbose=True,  # Show detailed unraveling steps
    indent=0
)

print(f"\nResult: {len(tokens)} tokens")
print(f"Text: {' '.join(tokens[:50])}...")  # Show first 50 tokens

## Cleanup

Close all connections when done.

In [ ]:
# # Close KATO client connections
# for i, node in enumerate(nodes):
#     node.close()
#     print(f"✓ Closed node{i}")

# print("\n✓ All KATO connections closed")

## Summary

**What you learned in this notebook:**

1. **KATO API Usage**:
   - `clear_stm()` - Clear short-term memory
   - `observe(strings=[...])` - Send single observation
   - `observe_sequence(observations=[...])` - Send batch of observations
   - `get_predictions()` - Get predictions from current STM

2. **Direct Database Pattern Retrieval**:
   - Direct queries from ClickHouse + Redis
   - Pattern structure: `pattern_data = [[child1], [child2], ...]`
   - Frequency statistics and pattern inspection

3. **Hierarchical Generation**:
   - **Bottom-up activation**: Input text activates patterns at all levels
   - **Top-down unraveling**: High-level patterns recursively expand to tokens
   - **Cascading constraints**: Each level constrains the levels below

4. **Key Concepts**:
   - Each token = one event (KATO ordering constraint)
   - Fallback logic for novel inputs
   - Recursive pattern unraveling
   - Faithful generation (patterns from training data)

**Next Steps**:
- Experiment with different inputs
- Adjust `CHUNK_SIZE` and see how it affects results
- Try generation from different hierarchical levels
- Inspect patterns to understand what the system learned

---

**Educational Goal Achieved**: You now understand how to use KATO's API for hierarchical text generation!

Use node3 pattern: 
b79d1353b9dd97e613e695042a4bb91bb7cc2ca9
for a nice hierarchical graph